In [ ]:
!pip install torch transformers sentence-transformers numpy nltk spacy

In [ ]:
import json
import spacy
from collections import deque
from transformers import BlenderbotTokenizer, BlenderbotForConditionalGeneration

# Cargar el modelo de Blenderbot
model_name = "facebook/blenderbot-400M-distill"
tokenizer = BlenderbotTokenizer.from_pretrained(model_name)
model = BlenderbotForConditionalGeneration.from_pretrained(model_name)

# Cargar intents
with open("intents.json", "r", encoding="utf-8") as file:
    intents = json.load(file)

# Cargar datos de celulares
with open("celulares_limpios.json", "r", encoding="utf-8") as file:
    celulares = json.load(file)

# Memoria del chatbot
memoria_chat = deque(maxlen=5)
preferencias_usuario = {}
inicio_recoleccion = False
preguntas_flujo = [
    {"clave": "bateria", "pregunta": "¿Te interesa que tenga una batería de larga duración?"},
    {"clave": "presupuesto", "pregunta": "¿Cuánto es tu presupuesto máximo?"},
    {"clave": "marca", "pregunta": "¿Qué marca prefieres?"},
    {"clave": "camara", "pregunta": "¿Quieres una buena cámara?"},
    {"clave": "rendimiento", "pregunta": "¿Necesitas un celular con buen rendimiento?"}
]
indice_pregunta = 0

# Cargar modelo de lenguaje para extraer palabras clave
nlp = spacy.load("es_core_news_sm")

def extraer_palabras_clave(texto):
    doc = nlp(texto)
    palabras_clave = [token.lemma_ for token in doc if token.pos_ in ["NOUN", "ADJ", "VERB", "NUM"]]
    return palabras_clave

def buscar_intent(mensaje):
    mensaje_procesado = " ".join(extraer_palabras_clave(mensaje))
    for intent in intents["intents"]:
        for pattern in intent["patterns"]:
            if pattern.lower() in mensaje_procesado.lower():
                return intent["responses"]
    return None

def generar_respuesta(mensaje):
    memoria_chat.append(mensaje)
    mensaje_contexto = " ".join(memoria_chat)
    inputs = tokenizer([mensaje_contexto], return_tensors="pt")
    reply_ids = model.generate(**inputs)
    respuesta = tokenizer.decode(reply_ids[0], skip_special_tokens=True)
    return respuesta

def recomendar_celular():
    celulares_filtrados = celulares
    for clave, valor in preferencias_usuario.items():
        if clave == "presupuesto":
            celulares_filtrados = [c for c in celulares_filtrados if c["precio"] <= valor]
        elif clave == "camara":
            celulares_filtrados = [c for c in celulares_filtrados if c["camara"] >= valor]
        elif clave == "rendimiento":
            celulares_filtrados = [c for c in celulares_filtrados if c["ram"] >= valor]
        elif clave == "bateria":
            celulares_filtrados = [c for c in celulares_filtrados if c["bateria"] >= valor]
        elif clave == "marca":
            celulares_filtrados = [c for c in celulares_filtrados if valor.lower() in c["marca"].lower()]
    return celulares_filtrados[0] if celulares_filtrados else None

def chatbot():
    global inicio_recoleccion, preferencias_usuario, indice_pregunta
    print("Hola, ¿en qué puedo ayudarte?")
    while True:
        user_input = input("Tú: ")
        if user_input.lower() == "salir":
            print("Chatbot: ¡Hasta luego!")
            break
        
        if not inicio_recoleccion:
            intent_respuesta = buscar_intent(user_input)
            if intent_respuesta:
                print(f"Chatbot: {intent_respuesta[0]}")
            elif "recomendar" in user_input.lower() or "celular" in user_input.lower():
                inicio_recoleccion = True
                indice_pregunta = 0
                preferencias_usuario.clear()
                print("Chatbot: ¡Entendido! Dime tus preferencias y te recomendaré un celular.")
                print(f"Chatbot: {preguntas_flujo[indice_pregunta]['pregunta']}")
            else:
                print(f"Chatbot: {generar_respuesta(user_input)}")
            continue
        
        clave_pregunta_actual = preguntas_flujo[indice_pregunta]["clave"]
        
        if "sí" in user_input.lower() or "claro" in user_input.lower():
            preferencias_usuario[clave_pregunta_actual] = 4000 if clave_pregunta_actual == "bateria" else 1
        elif "no" in user_input.lower():
            pass
        else:
            palabras_clave = extraer_palabras_clave(user_input)
            for palabra in palabras_clave:
                if palabra.isdigit():
                    preferencias_usuario[clave_pregunta_actual] = int(palabra)
                else:
                    preferencias_usuario[clave_pregunta_actual] = palabra
        
        indice_pregunta += 1
        if indice_pregunta < len(preguntas_flujo):
            print(f"Chatbot: {preguntas_flujo[indice_pregunta]['pregunta']}")
        else:
            celular_recomendado = recomendar_celular()
            if celular_recomendado:
                respuesta = f"Te recomiendo el {celular_recomendado['Model Name']} que cuesta ${celular_recomendado['Launched Price (USA)']}, tiene una cámara de {celular_recomendado['Front Camera (MP)']}MP y una cámara trasera de{celular_recomendado['Back Camera (MP)']}."
            else:
                respuesta = "No encontré un celular que cumpla con todas tus preferencias."
            print(f"Chatbot: {respuesta}")
            inicio_recoleccion = False
            preferencias_usuario.clear()
            indice_pregunta = 0

chatbot()

Hola, ¿en qué puedo ayudarte?
Chatbot: ¡Hola! ¿Cómo puedo ayudarte?
Chatbot: ¡Hola! ¿Cómo puedo ayudarte?
Chatbot: ¡Hola! ¿Cómo puedo ayudarte?
Chatbot:  Have you ever been to Buen Duel? It's a Mexican soccer tournament.
Chatbot:  Have you ever been to Buen Diego? It's an island in the Caribbean Sea.
Chatbot:  Have you ever been to Buenos Divide? It's a city in Las Vegas, Nevada.
Chatbot:  Have you ever been to Buenos Divide? It's a city in Las Vegas, Nevada.
Chatbot: ¡Hola! ¿Cómo puedo ayudarte?
Chatbot: ¡Entendido! Dime tus preferencias y te recomendaré un celular.
Chatbot: ¿Te interesa que tenga una batería de larga duración?
Chatbot: ¿Cuánto es tu presupuesto máximo?
Chatbot: ¿Qué marca prefieres?
Chatbot: ¿Quieres una buena cámara?
Chatbot: ¿Necesitas un celular con buen rendimiento?


KeyError: 'precio'